In [94]:
import os
import json
import random

root_dir = '../MATH'  # Replace with your directory
num_files_to_load = 1000  # Replace with number of files you want to pick

# Set a seed for randomness
random.seed(123)

json_files = []

# Search for json files in sub-folders of root directory
for dirpath, _, filenames in os.walk(root_dir):
    for f in filenames:
        if f.endswith('.json'):
            json_files.append(os.path.join(dirpath, f))

# Randomly select files
files_to_load = random.sample(json_files, num_files_to_load)

data = []

# Load content from files
for filename in files_to_load:
    with open(filename, 'r') as f:
        data.append(json.load(f))

print(data)

[{'problem': 'Abby, Bart, Cindy and Damon weigh themselves in pairs. Together Abby and Bart weigh 160 pounds, Bart and Cindy weigh 180 pounds, and Cindy and Damon weigh 200 pounds. How many pounds do Abby and Damon weigh together?', 'level': 'Level 2', 'type': 'Algebra', 'solution': 'Let the weights of Abby, Bart, Cindy, and Damon be $a$, $b$, $c$, and $d$, respectively. We have the equations \\begin{align*}\na+b&=160\\\\\nb+c&=180\\\\\nc+d&=200\n\\end{align*} Subtracting the second equation from the first, we have $(a+b)-(b+c)=160-180 \\Rightarrow a-c=-20$. Adding this last equation to the third given equation, we have $(a-c)+(c+d)=-20+200 \\Rightarrow a+d=180$. Thus, Abby and Damon together weigh $\\boxed{180}$ pounds.'}, {'problem': 'In the North Area Mall, 18 of the 90 stores sell shoes. If that same ratio holds true for the University Mall and 9 stores there sell shoes, how many stores are at University Mall?', 'level': 'Level 3', 'type': 'Prealgebra', 'solution': 'The ratio of sh

In [97]:
extracted_data = []
converted_data = []

for datum in data:
    problem = datum.get('problem')
    solution = datum.get('solution')
    # Extract the boxed solution
    boxed_solution = ""
    if "\\boxed{" in solution:
        start = solution.index("\\boxed{") + len("$\boxed{")
        end = solution.index("}", start)
        boxed_solution = solution[start:end]
    extracted_data.append({'problem': problem, 'boxed_solution': boxed_solution})
    
# Check if only numbers and periods in boxed_solution, convert to float
for item in extracted_data:
    boxed_solution = item['boxed_solution']
    if all(c.isdigit() or c == '.' for c in boxed_solution):
        item['boxed_solution'] = float(boxed_solution)
        converted_data.append(item)

print(converted_data)

[{'problem': 'Abby, Bart, Cindy and Damon weigh themselves in pairs. Together Abby and Bart weigh 160 pounds, Bart and Cindy weigh 180 pounds, and Cindy and Damon weigh 200 pounds. How many pounds do Abby and Damon weigh together?', 'boxed_solution': 180.0}, {'problem': 'In the North Area Mall, 18 of the 90 stores sell shoes. If that same ratio holds true for the University Mall and 9 stores there sell shoes, how many stores are at University Mall?', 'boxed_solution': 45.0}, {'problem': 'How many ways are there to put five beads on a necklace if there are eight distinct beads to choose from, and rotations and reflections of the necklace are considered the same?', 'boxed_solution': 672.0}, {'problem': 'We have a rectangle of dimensions $x - 2$ by $2x + 5$ such that its area is $8x - 6$. What is the value of $x$?', 'boxed_solution': 4.0}, {'problem': 'Using the letters $A$ and $B$, the following two-letter code words can be formed: $AA$, $AB$, $BB$, $BA$. Using the letters $A$, $B$, and 

In [98]:
with open('sanitized_data.jsonl', 'w') as outfile:
    for entry in converted_data:
        json.dump(entry, outfile)
        outfile.write('\n')